In [36]:
import pandas as pd
import pickle
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

In [15]:
Cell_data = pd.read_parquet('data/CycleMeetsExpressions/cell_data_full.parquet')

In [16]:
Cell_data.head(5)

,cellid,Phase,pseudotime,pseudotime_proxy,ENSMUSG00000000001,ENSMUSG00000000028,ENSMUSG00000000031,ENSMUSG00000000037,ENSMUSG00000000049,ENSMUSG00000000056,...,ENSMUSG00000118436,ENSMUSG00000118445,ENSMUSG00000118449,ENSMUSG00000118454,ENSMUSG00000118505,ENSMUSG00000118506,ENSMUSG00000118519,ENSMUSG00000118537,ENSMUSG00000118550,sum
0,SCG0088_AAACAGCCACCAACCG-1,G1,G1_1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0
1,SCG0088_AAACAGCCATGATTGT-1,G1,G1_13,13.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,211.0
2,SCG0088_AAACATGCACAACCTA-1,G1,G1_12,12.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,270.0
3,SCG0088_AAACATGCAGTCTAGC-1,G1,G1_3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,362.0
4,SCG0088_AAACATGCATGGCCCA-1,G2M,G2M_9,41.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,202.0


In [25]:
Cell_data = Cell_data.dropna(subset=['Phase'])

In [32]:
genes = list(Cell_data.columns[4::])
phases = set(Cell_data['Phase'].unique())
pseudotime = set(Cell_data['pseudotime_proxy'])

In [40]:
phases

{'G1', 'G2M', 'S'}

In [46]:
# seperate input and output variables 
X = Cell_data[genes]

In [47]:
y = Cell_data['Phase'].replace({'G1':1, 'S': 2, 'G2M':3})


/var/folders/5w/2zkqr5s90sn2973m1948xlw80000gn/T/ipykernel_3998/3318556569.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = Cell_data['Phase'].replace({'G1':1, 'S': 2, 'G2M':3})


In [37]:
# initialize a RF model 
model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

In [51]:
feat_selector = BorutaPy(
	verbose=2,
	estimator=model,
	n_estimators='auto',
	max_iter=10  # number of iterations to perform
)

In [ ]:
# Train Boruta
feat_selector.fit(X, y)

In [ ]:
len(feat_selector.support_)

In [ ]:

# print support and ranking for each feature
print('\n------Support and Ranking for each feature------')
for i in range(len(feat_selector.support_)):
	if feat_selector.support_[i]:
		print('Passes the test: ', X.columns[i],
			  ' - Ranking: ', feat_selector.ranking_[i])
	else:
		print('Doesn\'t pass the test: ',
			  X.columns[i], ' - Ranking: ', feat_selector.ranking_[i])

In [ ]:
# Read dict with cell phase decoding
with open ('data/CycleMeetsExpressions/pseudotime_dict.pickle', 'rb') as f:
	phase_dict = pickle.load(f)